함수 호출

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import cmath
import class_informs as inform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import math
import random as ran
from scipy.stats import truncnorm
import seaborn as sbn

ImportError: No module named 'torch'

데이터 정제

In [2]:
#timeseries data
all_data=pd.read_csv('../data/Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

dic_data = {k: v for k, v in all_data.groupby('date')}

#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('../data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])

dic_data[list(dic_data.keys())[0]]['index']=list(dic_data[list(dic_data.keys())[0]].index)

bw=list(dic_data[list(dic_data.keys())[0]]['bw'])
bw=np.array(bw)

beta=list(dic_data[list(dic_data.keys())[0]]['beta'])
beta=np.array(beta)

alpha=list(dic_data[list(dic_data.keys())[0]]['as'])
alpha=np.array(alpha)
#alpha=alpha.reshape(1,len(bw))

sector_set = set(list(dic_data[list(dic_data.keys())[0]]['sector']))
sector_list = list(dic_data[list(dic_data.keys())[0]]['sector'])

gbs=dic_data[list(dic_data.keys())[0]][['sector', 'index']].apply(tuple, axis=1)
group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

mq_list=list(dic_data[list(dic_data.keys())[0]]['mq'])
mq_set=set(list(dic_data[list(dic_data.keys())[0]]['mq']))

gbm=dic_data[list(dic_data.keys())[0]][['mq', 'index']].apply(tuple, axis=1)
group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

nbasset=len(bw)
nbsector=len(sector_set)
nbmq=len(mq_set)


using_dic = dic_data[pd.to_datetime(date_list[0])]

asset_list = []
for i in using_dic["sedol"]:
    asset_list.append(i)


dic_sedol_as = {using_dic["sedol"][i] : 10000*using_dic["as"][i] for i in using_dic.index}

dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }
dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

for i in using_dic.index:
    dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
    dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])


sedol_var_list =[]
for i in risk_sedol:
    sedol_var_list.append("d"+str(i))

for i in risk_sedol:
    sedol_var_list.append("q"+str(i))

sedol_var_list.append("assum")

alpha = []
for i in risk_sedol:

    alpha.append(-1*dic_sedol_as[i])

qmat=[]
for i in range(len(risk_mat)):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(10000*j)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

for i in range(len(risk_mat)):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

for i in range(1):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

q_con1 = []
q_con2 = []
q_val = []
for i in range(len((risk_mat[0]))):
    for j in range(len(risk_mat[0])):
        if j >= i:
            q_con1.append(i)
            q_con2.append(j)
            if i == j:
                ex_list = list(risk_mat[i])
                q_val.append(0.5*ex_list[j]*10000)
            else:
                ex_list = list(risk_mat[i])
                q_val.append(ex_list[j]*10000)

Q_con = []
Q_con.append(q_con1)
Q_con.append(q_con2)
Q_con.append(q_val)

In [46]:
asset_list[::-1]

['BZBY209',
 'BZ2Y3W7',
 'BZ0P3Z5',
 'BYZGYC0',
 'BYXD7B3',
 'BYXB201',
 'BYX4D52',
 'BYWTW73',
 'BYVY8G0',
 'BYV2325',
 'BYQQ3P5',
 'BYP66M4',
 'BYNF418',
 'BYND5W0',
 'BYND5N1',
 'BY9D546',
 'BY7QL61',
 'BWSW5J5',
 'BVFMFG8',
 'BTN1Y44',
 'BTN1Y11',
 'BSPHGL4',
 'BMH4NJ8',
 'BLNN369',
 'BJWG213',
 'BFG3KF2',
 'BF6V3T1',
 'BF0HZC2',
 'BF0FM92',
 'BDHLTQ5',
 'BD95VZ8',
 'BD5JS97',
 'BBM4S68',
 'B9M2WX3',
 'B92RRW2',
 'B8KQN82',
 'B8DMK08',
 'B84LSN3',
 'B840261',
 'B7VD3F2',
 'B7QQYV9',
 'B76FBQ0',
 'B6WVMH3',
 'B6TFK30',
 'B63QTN2',
 'B633030',
 'B5BT0K0',
 'B5BKPQ4',
 'B4NH4W5',
 'B40K911',
 'B3Q2FJ4',
 'B3KFWW1',
 'B28V183',
 'B1Z77F6',
 'B1L60G9',
 'B1FLZ21',
 'B1F76F9',
 'B198391',
 'B198380',
 'B142VX8',
 'B12GHV2',
 'B0SRLH6',
 'B0SRLF4',
 'B0J7D57',
 'B0F0F09',
 'B08G9Z7',
 'B067BM3',
 'B01R311',
 'B011WL6',
 'B00PWV9',
 '8066051',
 '2989828',
 '2989301',
 '2985677',
 '2983604',
 '2967181',
 '2965839',
 '2963899',
 '2963372',
 '2960384',
 '2958936',
 '2954019',
 '2947956',
 '29

함수 정의

In [3]:
def cal_obj(sol, risk_mat, alpha) :
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, alpha)
    obj = s_r+s_a
    return obj*10000


def select_candi(sol) :
    diff_add = np.array([[0]*len(alpha)]*len(alpha), float)
    
    pre = cal_obj(sol, risk_mat, alpha)
    
    sol = np.array(sol)
#     a = np.array(sol[sol>=0.001])
#     aa = np.sort(a)[::-1]
    not_zero_sol = np.argsort(sol)[::-1][:len(sol[sol>=0.001])]
    zero_sol = np.argsort(sol)[:len(sol[sol<0.001])]
    
    for i in not_zero_sol :
        w = sol[i]
        sol[i] = 0
        for j in zero_sol :
            w1 = sol[j]
            sol[j] = w
            diff_add[i][j] = cal_obj(sol, risk_mat, alpha) - pre
            sol[j] = w1
        sol[i] = w

    row, col = np.where(diff_add == np.nanmin(diff_add[diff_add>0]))
    sol[row] = 0
    sol[col] = 0.1
    
    candi = np.argsort(sol)[::-1][:len(sol[sol>=0.001])]
    
    return candi


def update(sol,timelimit) :
    start_time = time.time()
    ex = inform.informs(10000)
    ex.set_omega(risk_mat)
    
    sol_list = []
    for k in risk_sedol :
        sol_list.append(sol[k])
    result_list = []
    
    while(True) :
        if time.time() - start_time > timelimit:
            break
        con = select_candi(sol_list)
        ex.set_con(cons=con)
        sol_list2 = []
        try:
            sol2 = ex.solve(0.01)
            if sol2[0] != 0:
                for k in risk_sedol :
                    sol_list2.append(sol2[0][k])
                sol_list = sol_list2
                result_list.append(sol2[2])
            else:
                print("infeasible solution")
        except:
            print("no solution")
            
    return result_list, sol_list

def sample_Z(m, n):
    return Variable(torch.Tensor(np.random.uniform(0., 1., size=[m, n])))



        
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size=size, scale=xavier_stddev)


class Generator(nn.Module):
    def __init__(self,  in_dim=10, hd_dim=50, out_dim=nbasset):
        super(Generator, self).__init__()
        
        self.G_W1 = nn.Parameter(torch.from_numpy(xavier_init([in_dim, hd_dim])).float())
        self.G_b1 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
        self.G_W2 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, out_dim])).float())
        self.G_b2 = nn.Parameter(torch.from_numpy(np.zeros(shape=[out_dim])).float())

    def forward(self, x):
        x = F.relu( x @ self.G_W1 + self.G_b1)
        x = x @ self.G_W2 + self.G_b2
        return F.sigmoid(x)


class Discriminator(nn.Module):
    def __init__(self, in_dim=nbasset,train=False):
        super(Discriminator, self).__init__()
        self.tot_infs = 0.0
        self.tot_infs_nor = 0.0
        self.train = train
        self.x_threshold = 0.001
        self.inf_scale = 100.0
        self.Zstar = 1000
        
        self.in_dim = in_dim

        np.random.seed(0)
        self.Omega = Variable(torch.from_numpy(risk_mat)).float()

        self.card_upper = 50.
        self.card_lower = 70.
        
        
    def forward(self, x):
        tot_infs = Variable(torch.zeros(x.size()[0]))
        x_bw = Variable(torch.from_numpy(bw).float())
        alp = Variable(torch.Tensor(alpha).float())   
        bet = Variable(torch.Tensor(beta).float())
        
        # (3)
        #tot_infs += F.relu()
 
        # (4)
        tot_infs += F.relu(1. - torch.sum(x,1))
        tot_infs += F.relu(torch.sum(x,1) - 1.)
        
        # (5)
        tot_infs += F.relu(torch.abs(x-x_bw)@ Variable(torch.ones(self.in_dim))-0.05)
        
        
        # (6)
        for j in range(nbsector):
            k= (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_sector[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (7)
        for j in range(nbmq):
            k = (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_mq[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (8)
        k = (x - x_bw)@bet
        l = torch.abs(k)
        tot_infs += F.relu(l - 0.1)
        
        # (9) lb <= card(x) <= b
        if self.train:
            num_non_zeros = torch.sum(torch.tanh(x / self.x_threshold), 1)
            tot_infs += F.relu(num_non_zeros - self.card_upper) + F.relu(self.card_lower - num_non_zeros)
        else:
            num_non_zeros = torch.sum(x.data>self.x_threshold, 1).float()
            tot_infs += Variable(np.maximum(num_non_zeros - self.card_upper, 0) + np.maximum(self.card_lower - num_non_zeros, 0))
    
        
        # (10)
        l = 0.5*torch.sum(torch.abs(x - x_bw))
        tot_infs += F.relu(0.6 - l)
        #tot_infs += F.relu(l - 1.)
    
        
        # (11) a<=dQd <= b
        dQ = (x-x_bw) @ self.Omega
        size_x = x.size()
        dQd = torch.bmm(dQ.view(size_x[0],1,size_x[1]), (x-x_bw).view(size_x[0],size_x[1],1)).squeeze(1).squeeze(1)
        tot_infs += F.relu(dQd - 0.01)
        tot_infs += F.relu(0.0025 - dQd)
        
        
        # min dQd + ad (<= Zstar)
        l = (x-x_bw)@alp
        tot_infs += F.relu(dQd - l - self.Zstar)
        self.objs = dQd - l

            
        fea_probs = F.relu(1. - F.tanh(tot_infs/self.inf_scale))
        
        
        return fea_probs

def GAN(rotation):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    G = Generator(Z_dim, HL_dim, X_dim)

    sample_D = Discriminator(X_dim, train=False)
    train_D = Discriminator(X_dim, train=True)

    G_solver = optim.Adam(G.parameters(), lr=1e-3)


    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 1000 == 0:
            sample_noise = sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
    return best_sols, sols,fea_sample_obj_values

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(sols))]

    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in range(len(card_list[i])):
            card_freq[card_list[j]]+=1

    best70=np.argsort(sum(sols))[::-1][:70]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best70,bestsol



In [19]:
# best_sols, sols,fea_sample_obj_values=GAN(10000)

# best70

In [85]:
# sol={}
# for i in range(len(risk_sedol)):
#     sol[risk_sedol[i]]=best_sols[0][i]

# sol

초기값 설정

In [4]:
init_set='cplex'
if init_set=='cplex':
    sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000,time_init=30.0)[0]
    
elif init_set=='GAN':
    best_sols, sols,fea_sample_obj_values=GAN(10000)
    best70,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)
    
    sol={}
    for i in range(len(risk_sedol)):
        sol[risk_sedol[i]]=best_sols[0][i] 
    

CPXPARAM_TimeLimit                               30
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIQCP Presolve eliminated 1486 rows and 2 columns.
MIQCP Presolve modified 1476 coefficients.
Aggregator did 58 substitutions.
Reduced MIQCP has 1941 rows, 2839 columns, and 197656 nonzeros.
Reduced MIQCP has 492 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQCP has 2 quadratic constraints.
Presolve time = 0.16 sec. (431.54 ticks)
Probing time = 0.04 sec. (21.83 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.04 sec. (39.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0  

In [86]:
# best70,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)

# sol={}
# dd=0
# for i in range(len(risk_sedol)):
#     if i in best70:
#         sol[risk_sedol[i]]=1
#         dd+=1
#     else:
#         sol[risk_sedol[i]]=0
        
# dd

선택 자산 update

In [5]:
result = update(sol,50)

print("objective value 변화")
print(result[0])

0
1
no solution
2
3
no solution
4
5
no solution
6
7
no solution
8
9
no solution
10
11
no solution
12
13
no solution
14
15
no solution
16
17
no solution
18
19
no solution
20
21
no solution
22
23
no solution
24
25
no solution
26
27
no solution
28
29
no solution
30
31
no solution
objective value 변화
[]


In [70]:
dd=sol.keys()

In [73]:
ee=sol.keys()

In [90]:
sol

{'2000019': 0.0,
 '2002305': 0.013664956549150828,
 '2002479': 0.0,
 '2005973': 0.0,
 '2007849': 0.0,
 '2008154': 0.0,
 '2011602': 0.0,
 '2017213': 0.0,
 '2018175': 0.0,
 '2019479': 0.0,
 '2019952': 0.0,
 '2020459': 0.0,
 '2021690': 0.0,
 '2023607': 0.013060367234033591,
 '2023737': 0.0,
 '2023748': 0.0,
 '2024020': 0.0,
 '2026082': 0.0,
 '2026242': 0.0,
 '2026361': 0.0,
 '2027104': 0.0,
 '2027342': 0.022441205029385857,
 '2028572': 0.0,
 '2029809': 0.0,
 '2032067': 0.0,
 '2032380': 0.0,
 '2033004': 0.017781139371923514,
 '2034494': 0.0,
 '2034524': 0.0,
 '2038430': 0.0,
 '2039831': 0.0,
 '2041364': 0.0,
 '2043694': 0.0,
 '2043962': 0.0,
 '2044545': 0.017906201886499426,
 '2046251': 0.01141657954301406,
 '2046552': 0.0,
 '2047317': 0.0,
 '2048804': 0.0,
 '2050832': 0.0,
 '2057059': 0.0,
 '2065159': 0.0,
 '2065308': 0.0,
 '2065955': 0.0,
 '2066408': 0.0,
 '2066721': 0.0,
 '2068921': 0.0,
 '2069065': 0.0,
 '2073022': 0.0,
 '2073408': 0.0,
 '2077905': 0.0,
 '2081779': 0.0,
 '2084637': 0.0